# O3-Mini Acuity Prediction 
This notebook predicts ER acuity level using the O3-Mini model, with all demographic inputs included.

In [ ]:
# 📦 Imports and Setup
import pandas as pd
import time, os, requests
from dotenv import load_dotenv
load_dotenv()

# Load dataset
df = pd.read_csv('acuity_results.csv')

In [ ]:
# 🔐 Azure API Configuration
API_KEY = os.getenv('API_KEY')
API_VERSION = '2024-12-01-preview'
RESOURCE_ENDPOINT = os.getenv('RESOURCE_ENDPOINT')
DEPLOYMENT_NAME = 'o3-mini-2025-01-31'

In [ ]:
# 🧠 Prompt Generator Function 
def format_prompt(row):
    prompt = "Predict the emergency department acuity level for this patient.\n\n"
    if 'primarychiefcomplaintname' in row:
        prompt += f"Chief Complaint: {row['primarychiefcomplaintname']}\n"
    if 'age' in row:
        prompt += f"Age: {row['age']}\n"
    if 'sex' in row:
        prompt += f"Sex: {row['sex']}\n"
    if 'firstrace' in row:
        prompt += f"Race: {row['firstrace']}\n"
    if 'Vital_Signs' in row:
        prompt += f"Vital Signs: {row['Vital_Signs']}\n"
    prompt += "\nSelect the most appropriate acuity level from the following options ONLY:\n"
    prompt += "'Immediate', 'Emergent', 'Urgent', 'Less Urgent', 'Non-Urgent'\n\n"
    prompt += "Respond with ONLY ONE of these five options."
    return prompt

In [ ]:
# 🤖 Query O3-Mini Model
def call_o3(prompt):
    url = f"{RESOURCE_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"
    headers = {
        'Content-Type': 'application/json',
        'api-key': API_KEY
    }
    payload = {
        'messages': [
            {"role": "system", "content": prompt},
        ],
    }
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=30)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
# 🧪 Run Prediction
results = []
for _, row in df.iterrows():
    prompt = format_prompt(row)
    prediction = call_o3(prompt)
    results.append(prediction)
    time.sleep(1)  # Be nice to the API

df['o3_prediction'] = results
df.to_csv('o3_acuity_single_experiment.csv', index=False)
df[['primarychiefcomplaintname', 'o3_prediction']].head()